In [ ]:
import os

# UU : EW
# VV : NS
def load_displacements(ica_path):
    uu, vv = [], []
    for filename in os.listdir(ica_path):
        if 'FILTER_Displmax' in filename:
            raster = gdal.Open(ica_path + filename)
            values = raster.GetRasterBand(1).ReadAsArray()
            if 'MM_NS' in filename:
                vv.append(values)
            if 'MM_EW' in filename:
                uu.append(values)
    return np.array(uu), np.array(vv)


def get_component_filenames(ica_path, component):
    filenames = []
    for filename in os.listdir(ica_path):
        if 'FILTER_Displmax' filename and  component in filename:
            filenames.append(filename)
    return filenames


# chargement des données
ica_path = ''
UU, VV = load_displacements(ica_path)
filenames = get_component_filenames(ica_path, 'MM_NS') # je compte juste le nombre de rasters avec MM_NS


# calcul de la matrice A
A = []
n_filenames = len(filenames) # c'est peut-être incorrect !
for i in range(0, n_filenames - 1):
    for j in range(i+1, n_filenames):
        b = np.zeros(n_filenames -1)
        b[i:j] = 1
        A.append(b)
AA = np.linalg.inv(np.transpose(A).dot(A)).dot(np.transpose(A))


# transformation des données pour ICA
vv_4ica=[]
uu_4ica=[]
for i in range(0, np.size(VV,1)):
    for j in range(0, np.size(UU,2)):
            vv_4ica.append(VV[:,i,j])
            uu_4ica.append(UU[:,i,j]) 
            
# ica         
Nica=10
icav = FastICA(n_components=Nica, whiten='boolean')
icau = FastICA(n_components=Nica, whiten='boolean')

S_v = icav.fit_transform(vv_4ica)  # Reconstruct signals
S_u = icau.fit_transform(uu_4ica)  # Reconstruct signals
S_v_t = np.transpose(S_v)
S_u_t = np.transpose(S_u)
A_v = icav.mixing_
A_u = icau.mixing_

# inversion
du = []
dv = []
for i in range(0, 10):
    du.append(AA.dot(S_u_t[i, :]))
    dv.append(AA.dot(S_v_t[i, :]))